In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from scipy.stats import randint, uniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [11]:
df = pd.read_csv('Данные для построения модели, задача - Классификация - превышает ли значение SI медианное значение выборки.csv')
df

,EState_VSA4,FpDensityMorgan1,VSA_EState1,BalabanJ,HallKierAlpha,SlogP_VSA5,qed,SMR_VSA5,AvgIpc,FractionCSP3,...,PEOE_VSA8,TPSA,EState_VSA8,MinPartialCharge,MaxAbsEStateIndex,BCUT2D_MWLOW,BCUT2D_MRLOW,SlogP_VSA2,SMR_VSA10,Target Feature
0,24.925325,0.642857,0.000000,1.164038,-0.66,105.750639,0.417362,105.750639,3.150503,0.923077,...,35.342864,24.72,41.542423,-0.293526,5.094096,9.700470,0.193481,24.512883,11.423370,0
1,23.919494,0.607143,0.000000,1.080362,-0.08,105.750639,0.462473,117.834321,3.150503,1.000000,...,12.083682,24.06,52.176000,-0.313407,3.961417,9.689226,0.120322,25.173194,0.000000,0
2,23.919494,0.562500,2.517630,1.219066,-0.08,105.750639,0.260923,117.834321,3.214947,1.000000,...,23.671624,0.00,69.733111,-0.325573,2.627117,9.681293,-0.922902,62.329944,0.000000,0
3,24.925325,0.620690,0.000000,1.120513,-0.66,112.171461,0.377846,112.171461,3.179270,0.925926,...,35.342864,24.72,41.542423,-0.293526,5.097360,9.700497,0.193510,24.512883,11.423370,1
4,29.631406,0.600000,0.000000,1.136678,-2.22,91.194256,0.429038,86.488175,3.337074,0.575758,...,22.253351,24.72,90.073360,-0.257239,5.150510,9.700386,0.193687,11.423370,22.798143,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,38.524930,1.133333,10.503509,1.646946,-1.65,66.219879,0.382752,66.219879,3.023764,0.800000,...,5.414990,69.67,33.770969,-0.468587,12.934891,9.344314,-0.196385,25.017713,17.907916,0
997,38.524930,1.085714,10.086396,1.535171,-2.18,73.143616,0.369425,79.185457,3.130958,0.785714,...,10.829981,98.77,39.087758,-0.467493,13.635345,9.343622,-0.187625,36.966734,23.815096,0
998,44.277783,1.157895,10.305031,1.493776,-1.83,72.640700,0.284923,78.682541,3.204255,0.800000,...,10.829981,98.77,39.087758,-0.467485,13.991690,9.343613,-0.187687,48.975357,35.576981,0
999,6.420822,0.756757,20.885832,2.325807,-2.45,66.219879,0.381559,66.219879,2.887043,0.821429,...,11.835812,122.27,6.923737,-0.468755,13.830180,9.364015,-0.225309,58.099656,29.660466,0


In [12]:
X, y = df.drop(['Target Feature'], axis=1), df['Target Feature']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
y.value_counts()

Target Feature
0    893
1    108
Name: count, dtype: int64

Классы не сбалансированны, произведем процесс балансировки классов

In [15]:
smote = SMOTE(sampling_strategy='auto', k_neighbors=10, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [16]:
X, y = X_resampled, y_resampled
y.value_counts()

Target Feature
0    893
1    893
Name: count, dtype: int64

Классы сбалансированны, значит процесс балансировки классов нам не требуется

In [21]:
class BestClassifierModel:
    """
    Класс для отбора лучшей модели машинного обучения для модели классификации
    X: DataFrame с признаками
    y: Series с целевой переменной

    В данном классе идет подбор 4 разных моделей машинного обучения.
    Для каждой модели производиться подор гипперпараметров (если таковые имеются)
    После получения моделей с лучшими параметрами идет отбор лучшей модели по средству сравнения assessment_criterion с полученными метриками моделей
    """
    def __init__(self, X, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        self.assessment_criterion = {
            'metric_name': 'F1-score',
            'value': 0.7
        }
        self.best_model = None
        self.models = {
            'linear_regression': {
                'model': LogisticRegression(max_iter=1000, random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': [0.01, 0.1, 1, 10],
                    'penalty': ['l1', 'l2'],
                    'solver': ['saga']
                },
                'parameter_selection_method': 'GridSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'elastic_net': {
                'model': KNeighborsClassifier(),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'GridSearchCV',
                'params': {
                    'n_neighbors': range(3, 15),
                    'weights': ['uniform', 'distance'],
                    'p': [1, 2]  # 1 = манхэттенское расстояние, 2 = евклидово расстояние
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'random_forest_regressor': {
                'model': RandomForestClassifier(random_state=42),
                'best_model': None,
                'best_params': None,
                'parameter_selection_method': 'RandomizedSearchCV',
                'params': {
                    'n_estimators': randint(50, 200),
                    'max_depth': [None, 5, 10, 20],
                    'min_samples_split': randint(2, 10),
                    'min_samples_leaf': randint(1, 5)
                },
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': False,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                }
            },
            'randomized_search_cv_model': {
                'model': SVC(random_state=42),
                'best_model': None,
                'best_params': None,
                'params': {
                    'C': uniform(0.1, 10),
                    'kernel': ['linear', 'rbf'],
                    'gamma': ['scale', 'auto']
                },
                'parameter_selection_method': 'RandomizedSearchCV',
                'metrics': {
                    'Accurac': '',
                    'Precision': '',
                    'Recall': '',
                    'F1-score': '',
                },
                'standartization': True,
                'data': {
                    'X_train': self.X_train,
                    'X_test': self.X_test,
                    'y_train': self.y_train,
                    'y_test': self.y_test,
                    'y_pred': None
                }
            }
        }
        
    
    def fit(self):
        """
        Метод подбора гипперпараметров и инизиализации лучших моделей с подобранными гипперпараметрами для каждой модели
        """
        for model in self.models:
            print(f"Подбор гипперпараметров для модели {str(self.models[model]['model'])} методом {self.models[model]['parameter_selection_method']}")
            
            X, y = self.models[model]['data']['X_train'], self.models[model]['data']['y_train']
            
            if self.models[model]['standartization']:
                print('Перед подобром, необходом провести стандартизацию параметров')
                X = self.X_train_stnd = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаем к подбору гиппепараметров')
            print('Ожидайте...')
            
            if self.models[model]['parameter_selection_method'] == 'GridSearchCV':
                grid = GridSearchCV(self.models[model]['model'], self.models[model]['params'], cv=5, scoring='neg_mean_squared_error')
                grid.fit(X, y)
                self.models[model]['best_params'] = grid.best_params_
                self.models[model]['best_model'] = grid.best_estimator_
            elif self.models[model]['parameter_selection_method'] == 'RandomizedSearchCV':
                search = RandomizedSearchCV(self.models[model]['model'], self.models[model]['params'], n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)
                search.fit(X, y)
                self.models[model]['best_params'] = search.best_params_
                self.models[model]['best_model'] = search.best_estimator_
            print(f"Лучшие параметры для модели {str(self.models[model]['model'])}:", self.models[model]['best_params'])
            print()
                
    def get_best_model(self):
        """
        Метод получения лучшей модели по показателю assessment_criterion
        В данном методе происходит обучение моделей на тестовой выборке и получение метрик на основе полученных предсказаний
        
        По итогу работы модели мы получаем модель, которая имеет лучший показатель по критрию заданному в assessment_criterion или
        уведомление о том, что модель не была выбрана. Тогда необходимо либо изменить метрику, либо изменить значение метрики
        """
        for model in self.models:
            print(f"Обучаем модель {str(self.models[model]['model'])} с лучшими гипперпараметрами для модели:")
            
            X, y = self.models[model]['data']['X_test'], self.models[model]['data']['y_test']
            
            if self.models[model]['standartization']:
                print('Перед обучением, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(X) 
                print('К параметрам применен метод StandardScaler')
            
            print('Приступаем к формированию предсказания на тестовой выборке')
            
            self.models[model]['data']['y_pred'] = self.models[model]['best_model'].predict(X)
            
            self.models[model]['metrics']['Accurac'] = accuracy_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Precision'] = precision_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['Recall'] = recall_score(y, self.models[model]['data']['y_pred'])
            self.models[model]['metrics']['F1-score'] = f1_score(y, self.models[model]['data']['y_pred'])
            
            print(f"Метрики для модели {str(self.models[model]['model'])}:")
            print(f"Accuracy: {self.models[model]['metrics']['Accurac']:.4f}")
            print(f"Precision: {self.models[model]['metrics']['Precision']:.4f}")
            print(f"Recall: {self.models[model]['metrics']['Recall']:.4f}")
            print(f"F1-score: {self.models[model]['metrics']['F1-score']:.4f}")
            print()

            if self.models[model]['metrics'][self.assessment_criterion['metric_name']] > self.assessment_criterion['value']:
                self.assessment_criterion['value'] = self.models[model]['metrics'][self.assessment_criterion['metric_name']]
                self.best_model = self.models[model]
        
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована')
            print('Измените value или metric_name в assessment_criterion')
        else:
            print('Модель для формирования прогноза сформирована')
            print(f"Лучшая модель по главному критерию оценки {self.assessment_criterion['metric_name']}:")
            print(f"{str(self.best_model['best_model'])}:")
            print('Метрики')
            for k, v in self.best_model['metrics'].items():
                print(f'{k} {v:.4f}')
            
    
    def predict(self, data):
        """
        Метод для формирования прогноза с помощью лучшей модели, которая была получена в методе get_best_model
        """
        result = None
        X = data
        if self.best_model == None:
            print('Модель для формирования прогноза не сформирована, используйте метод get_best_model')
            return result
        else:
            print(f"Формируем предсказание с помощью модели {str(self.best_model['best_model'])}")
            if self.best_model['standartization']:
                print('Перед формированияем прогноза, необходом провести стандартизацию параметров')
                X = StandardScaler().fit_transform(data) 
                print('К параметрам применен метод StandardScaler')
                print('Приступаю формированию прогноза...')
            result = self.best_model['best_model'].predict(X)
            print('Предсказание сформировано')
        return result

In [22]:
best_regression_model = BestClassifierModel(X, y)
best_regression_model.fit()

Подбор гипперпараметров для модели LogisticRegression(max_iter=1000, random_state=42) методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...


g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_Folder\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
g:\PythonProject\Educational_F

Лучшие параметры для модели LogisticRegression(max_iter=1000, random_state=42): {'C': 10, 'penalty': 'l1', 'solver': 'saga'}

Подбор гипперпараметров для модели KNeighborsClassifier() методом GridSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели KNeighborsClassifier(): {'n_neighbors': 4, 'p': 1, 'weights': 'distance'}

Подбор гипперпараметров для модели RandomForestClassifier(random_state=42) методом RandomizedSearchCV
Ожидайте...
Лучшие параметры для модели RandomForestClassifier(random_state=42): {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 141}

Подбор гипперпараметров для модели SVC(random_state=42) методом RandomizedSearchCV
Перед подобром, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к подбору гиппепараметров
Ожидайте...
Лучшие параметры для модели SVC

In [23]:
best_regression_model.get_best_model()

Обучаем модель LogisticRegression(max_iter=1000, random_state=42) с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели LogisticRegression(max_iter=1000, random_state=42):
Accuracy: 0.8116
Precision: 0.9831
Recall: 0.6397
F1-score: 0.7751

Обучаем модель KNeighborsClassifier() с лучшими гипперпараметрами для модели:
Перед обучением, необходом провести стандартизацию параметров
К параметрам применен метод StandardScaler
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели KNeighborsClassifier():
Accuracy: 0.9198
Precision: 0.8962
Recall: 0.9522
F1-score: 0.9234

Обучаем модель RandomForestClassifier(random_state=42) с лучшими гипперпараметрами для модели:
Приступаем к формированию предсказания на тестовой выборке
Метрики для модели RandomForestClassifier(random_state=42):
Accuracy: 0.9851
Precisi

In [24]:
best_regression_model.predict(X_test)

Формируем предсказание с помощью модели RandomForestClassifier(min_samples_leaf=2, min_samples_split=5,
                       n_estimators=141, random_state=42)
Предсказание сформировано


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]